In [35]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision import datasets
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from tqdm.auto import tqdm
import random

# 指定需要的类别
selected_classes = [0, 1, 2, 3, 4]
label_mapping = {orig_label: idx for idx, orig_label in enumerate(selected_classes)}
# 获取 CIFAR-100 的类别映射
cifar100_classes = torchvision.datasets.CIFAR10(root='./data', download=False).classes

# 数据预处理
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# 加载 CIFAR-100 数据集
train_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=False, transform=transform_train
)

# 筛选出所需类别的索引
selected_indices_train_class = [
    idx for idx, (_, label) in enumerate(train_set)
    if label in selected_classes
]
random.shuffle(selected_indices_train_class)
selected_indices_train=[]
class_cnt=[0]*10
for i in selected_indices_train_class:
    if class_cnt[train_set.targets[i]]<1000:
        selected_indices_train.append(i)
        class_cnt[train_set.targets[i]]+=1

for i in selected_indices_train:
    train_set.targets[i]=label_mapping[train_set.targets[i]]


bt_size=128
# 创建子集数据集
filtered_train_set = Subset(train_set, selected_indices_train)

train_loader = DataLoader(filtered_train_set, batch_size=bt_size, shuffle=True, num_workers=2)




# 加载 CIFAR-100 测试集
test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=False, transform=transform_test
)


selected_indices_test = [
    idx for idx, (_, label) in enumerate(test_set)
    if label in selected_classes
]


for i in selected_indices_test:
    test_set.targets[i]=label_mapping[test_set.targets[i]]

# 创建子集数据集
filtered_test_set = Subset(test_set, selected_indices_test)
test_loader = DataLoader(filtered_test_set, batch_size=bt_size, shuffle=False, num_workers=2)

print('Finish data loading')
print(f"Training data size: {len(filtered_train_set)}")
print(f"Testing data size: {len(filtered_test_set)}")


Finish data loading
Training data size: 5000
Testing data size: 5000


In [36]:
""" def train(model, train_loader, epochs=10, device='cpu'):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    criterion = torch.nn.CrossEntropyLoss()# give y hat and y calculate the loss
    for i in range(epochs):

        loss_final = 0.0
        loop = tqdm(enumerate(train_loader), total=len(train_loader))
        loop.set_description(f'Epoch [{i}/{epochs}]')

        for step, (images, labels) in loop:

            
            images, labels = images.to(device), labels.to(device)
            # forward + backward + update
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_final += loss.item()
            loop.set_postfix(loss=loss_final)
        
        test(test_loader,model,device) """

def test(test_loader,model,device='cpu'):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:#包含batch size=64张图片和label
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)#predicted是一个tensor,torch.tensor([1, 2, 3, ..., 64])  # 共 64 个元素
            # 返回最大值和下标
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Accuracy on test set: %.3f %%' % (100 * correct / total))

In [37]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [43]:

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_mobilenetv2_x0_5", pretrained=False)
model.to(device)

Using cache found in /home/chunjie/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(8, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(48, eps=1e-05, momentum

In [44]:
def train(epoch):
    running_loss = 0.0 

    
    for batch_idx, data in enumerate(train_loader, 0):

        inputs, target = data
        inputs, target = inputs.to(device) , target.to(device)
        optimizer.zero_grad()
        #print(batch_idx)

        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print('epoch: %d loss:%.3f ' % (epoch,running_loss), end=' ')

In [45]:

if __name__ == "__main__":
    epoch=100
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()# give y hat and y calculate the loss
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)
    for i in range(epoch):
        train(i)
        scheduler.step()
        test(test_loader,model,device)

epoch: 0 loss:123.236  Accuracy on test set: 24.440 %
epoch: 1 loss:68.098  Accuracy on test set: 41.900 %
epoch: 2 loss:55.631  Accuracy on test set: 43.860 %
epoch: 3 loss:51.651  Accuracy on test set: 47.020 %
epoch: 4 loss:49.976  Accuracy on test set: 51.860 %
epoch: 5 loss:46.399  Accuracy on test set: 53.180 %
epoch: 6 loss:44.895  Accuracy on test set: 55.260 %
epoch: 7 loss:44.838  Accuracy on test set: 54.980 %
epoch: 8 loss:42.859  Accuracy on test set: 56.840 %
epoch: 9 loss:42.850  Accuracy on test set: 53.460 %
epoch: 10 loss:43.927  Accuracy on test set: 56.340 %
epoch: 11 loss:41.576  Accuracy on test set: 58.020 %
epoch: 12 loss:39.476  Accuracy on test set: 60.140 %
epoch: 13 loss:38.772  Accuracy on test set: 59.640 %
epoch: 14 loss:39.396  Accuracy on test set: 60.040 %
epoch: 15 loss:38.742  Accuracy on test set: 61.800 %
epoch: 16 loss:37.884  Accuracy on test set: 61.880 %
epoch: 17 loss:37.103  Accuracy on test set: 61.000 %
epoch: 18 loss:36.193  Accuracy on te

In [47]:
model.eval()
test(test_loader,model,device)


Accuracy on test set: 81.380 %


In [48]:
import os
save_dir = './model_CIFAR10'  # 目录
save_path = f'{save_dir}/model_org.pt'  # 文件路径

# 确保目录存在，但不要创建文件
os.makedirs(save_dir, exist_ok=True)  # ✅ 只创建目录，不影响文件

# 保存模型
torch.save(model.state_dict(), save_path)  # ✅ 保存文件
print(f"Model saved at: {save_path}")

Model saved at: ./model_CIFAR10/model_org.pt


In [52]:
model_test=torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_mobilenetv2_x0_5", pretrained=False)
model_test.to(device)
model_test.load_state_dict(torch.load(f'./model_CIFAR10/model_org.pt'))
model_test.eval()
test(test_loader,model_test,device)
test(train_loader,model_test,device)

Using cache found in /home/chunjie/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master
/tmp/ipykernel_1023834/717139178.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


Accuracy on test set: 81.380 %
Accuracy on test set: 92.420 %
